## Step 1: Prepare the Training and Evaluation Dataset

In [1]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
(X.shape, y.shape)

((442, 10), (442,))

In [2]:
[(X[0], y[0]), (X[1], y[1])]

[(array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
         -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613]),
  151.0),
 (array([-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
         -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405]),
  75.0)]

## Step 2: Launching the Spark Session with Rikai Extension

In [3]:
from example import spark

23/01/07 21:25:20 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.31.194 instead (on interface wlx1cbfce3ffbfe)
23/01/07 21:25:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/da/.cache/pants/named_caches/pex_root/installed_wheels/8f254bc20b539246427b2913639b8a0258db76ab54ba91fbbebb8dc8c36183c1/pyspark-3.3.1-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/da/.ivy2/cache
The jars for the packages stored in: /home/da/.ivy2/jars
ai.eto#rikai_2.1 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c3e25b0e-2ac6-4e11-974c-1b464c906669;1.0
	confs: [default]
:: resolution report :: resolve 14019ms :: artifacts dl 1ms
	:: modules in use:
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   0   |   0   |
	---------------------------------------------------------------------

:: problems summary ::
:::: WARNINGS
		module not found: ai.eto#rikai_2.1;0.2.0-SNAPSHOT

	==== local-m2-cache: tried

	  file:/home/da/.m2/repository/ai/eto/rikai_2.1/0.2.0-SNAPSHOT/rikai_2.1-0.2.0-SNAPSHOT.pom

	  -- artifa

RuntimeError: Java gateway process exited before sending its port number

## Step 3: Training and Logging using MLflow

In [ ]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.linear_model import LinearRegression


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
model = LinearRegression()
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)

    registered_model_name = f"{getpass.getuser()}_sklearn_lr"
    log_model(model, registered_model_name)


## Step 4: Create the model using the registered MLflow uri

In [ ]:
from liga.mlflow.logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


## Step 5: predict using the registered Rikai model

In [ ]:
from liga.numpy.sql import literal

result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, {literal(X[0])})
"""
)

result.printSchema()
result.toPandas()

In [ ]:
spark.sql(f"""
select  ML_PREDICT(mlflow_sklearn_m, {literal(X[1])})
""").toPandas()